### Import all sequences

In [4]:
#import sequences into qiime2
!qiime tools import \
    --type 'SampleData[PairedEndSequencesWithQuality]' \
    --input-format CasavaOneEightSingleLanePerSampleDirFmt \
    --input-path ../seqs/LibCombined \
    --output-path ../data/LibCombined/full-diversity-seqs.qza

Imported ../seqs/LibCombined as CasavaOneEightSingleLanePerSampleDirFmt to ../data/LibCombined/full-diversity-seqs.qza


In [5]:
!qiime demux summarize \
--i-data ../data/LibCombined/full-diversity-seqs.qza \
--o-visualization ../data/LibCombined/full-diversity-seqs.qzv

Saved Visualization to: ../data/LibCombined/full-diversity-seqs.qzv


In [4]:
!qiime tools view ../data/LibCombined/full-diversity-seqs.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.
Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

### Denoise the sequences and create OTU table

In [4]:
!qiime dada2 denoise-paired --help

Usage: qiime dada2 denoise-paired [OPTIONS]

  This method denoises paired-end sequences, dereplicates them, and filters
  chimeras.

Inputs:
  --i-demultiplexed-seqs ARTIFACT SampleData[PairedEndSequencesWithQuality]
                         The paired-end demultiplexed sequences to be
                         denoised.                                  [required]
Parameters:
  --p-trunc-len-f INTEGER
                         Position at which forward read sequences should be
                         truncated due to decrease in quality. This truncates
                         the 3' end of the of the input sequences, which will
                         be the bases that were sequenced in the last cycles.
                         Reads that are shorter than this value will be
                         discarded. After this parameter is applied there must
                         still be at least a 12 nucleotide overlap between the
                         forward and reverse reads. If 

In [7]:
# Trim primers off the beginning of the sequences where there is noise- to 13- and trim the ends of the 
#    sequences where the quality score at the 25th percentile drops below 35, which is at 189 for the fwd reads
#    and 161 for the rev reads.

!qiime dada2 denoise-paired --verbose \
  --i-demultiplexed-seqs ../data/LibCombined/full-diversity-seqs.qza \
  --o-table ../data/LibCombined/full-diversity-table \
  --o-representative-sequences ../data/LibCombined/full-diversity-rep-seqs \
  --output-dir ../data/LibCombined/full-diversity-stats \
  --p-n-threads 0 \
  --p-trim-left-f 13 \
  --p-trim-left-r 13 \
  --p-trunc-len-f 182 \
  --p-trunc-len-r 161 \

Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: run_dada_paired.R /var/folders/cb/6fpykkvj2pl0g6rq7_tjvms00000gn/T/tmp88fcmyh5/forward /var/folders/cb/6fpykkvj2pl0g6rq7_tjvms00000gn/T/tmp88fcmyh5/reverse /var/folders/cb/6fpykkvj2pl0g6rq7_tjvms00000gn/T/tmp88fcmyh5/output.tsv.biom /var/folders/cb/6fpykkvj2pl0g6rq7_tjvms00000gn/T/tmp88fcmyh5/track.tsv /var/folders/cb/6fpykkvj2pl0g6rq7_tjvms00000gn/T/tmp88fcmyh5/filt_f /var/folders/cb/6fpykkvj2pl0g6rq7_tjvms00000gn/T/tmp88fcmyh5/filt_r 182 161 13 13 2.0 2.0 2 independent consensus 1.0 0 1000000

R version 4.0.3 (2020-10-10) 
Loading required package: Rcpp
DADA2: 1.18.0 / Rcpp: 1.0.6 / RcppParallel: 5.0.2 
1) Filtering ...................................................................................................................................

In [8]:
!qiime feature-table summarize \
    --i-table ../data/LibCombined/full-diversity-table.qza \
    --o-visualization ../data/LibCombined/full-diversity-table.qzv 


Saved Visualization to: ../data/LibCombined/full-diversity-table.qzv


In [8]:
!qiime feature-table tabulate-seqs \
    --i-data ../data/LibCombined/full-diversity-rep-seqs.qza \
    --o-visualization ../data/LibCombined/full-diversity-rep-seqs.qzv

^C

Aborted!


In [10]:
!qiime tools view ../data/LibCombined/full-diversity-table.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.
Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [11]:
!qiime tools view ../data/LibCombined/full-diversity-rep-seqs.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.
Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

### Assign taxonomy

In [12]:
!qiime feature-classifier classify-sklearn \
  --i-classifier silva-138-99-515-806-nb-classifier.qza \
  --i-reads ../data/LibCombined/full-diversity-rep-seqs.qza \
  --o-classification ../data/LibCombined/full-diversity-taxonomy.qza 

Saved FeatureData[Taxonomy] to: ../data/LibCombined/full-diversity-taxonomy.qza


In [1]:
!qiime feature-classifier classify-sklearn --help

Usage: qiime feature-classifier classify-sklearn [OPTIONS]

  Classify reads by taxon using a fitted classifier.

Inputs:
  --i-reads ARTIFACT FeatureData[Sequence]
                         The feature data to be classified.         [required]
  --i-classifier ARTIFACT
    TaxonomicClassifier  The taxonomic classifier for classifying the reads.
                                                                    [required]
Parameters:
  --p-reads-per-batch VALUE Int % Range(1, None) | Str % Choices('auto')
                         Number of reads to process in each batch. If "auto",
                         this parameter is autoscaled to min( number of query
                         sequences / n-jobs, 20000).         [default: 'auto']
  --p-n-jobs INTEGER     The maximum number of concurrently worker processes.
                         If -1 all CPUs are used. If 1 is given, no parallel
                         computing code is used at all, which is useful for
                       

### Export files and create a .biom file

In [1]:
!mkdir ../data/LibCombined/full-diversity_OTU_table

In [2]:
!qiime tools export --input-path ../data/LibCombined/full-diversity-table.qza --output-path ../data/LibCombined/full-diversity_OTU_table
!qiime tools export --input-path ../data/LibCombined/full-diversity-rep-seqs.qza --output-path ../data/LibCombined/full-diversity_OTU_table
!qiime tools export --input-path ../data/LibCombined/full-diversity-taxonomy.qza --output-path ../data/LibCombined/full-diversity_OTU_table

Exported ../data/LibCombined/full-diversity-table.qza as BIOMV210DirFmt to directory ../data/LibCombined/full-diversity_OTU_table
Exported ../data/LibCombined/full-diversity-rep-seqs.qza as DNASequencesDirectoryFormat to directory ../data/LibCombined/full-diversity_OTU_table
Exported ../data/LibCombined/full-diversity-taxonomy.qza as TSVTaxonomyDirectoryFormat to directory ../data/LibCombined/full-diversity_OTU_table


In [3]:
## Copy sample data I think.

!cp ../temporary-metadata.csv ../data/LibCombined/full-diversity_OTU_table/

In [4]:
!ls ../data/LibCombined/full-diversity_OTU_table/

dna-sequences.fasta    taxonomy.tsv
feature-table.biom     temporary-metadata.csv


In [5]:
!biom add-metadata \
-i ../data/LibCombined/full-diversity_OTU_table/feature-table.biom \
-o ../data/LibCombined/full-diversity_OTU_table/feature-table-metaD.biom \
-m ../data/LibCombined/full-diversity_OTU_table/temporary-metadata.csv

In [6]:
!ls ../data/LibCombined/full-diversity_OTU_table/

dna-sequences.fasta      feature-table.biom       temporary-metadata.csv
feature-table-metaD.biom taxonomy.tsv


In [7]:
!biom add-metadata \
-i ../data/LibCombined/full-diversity_OTU_table/feature-table-metaD.biom \
-o ../data/LibCombined/full-diversity_OTU_table/feature-table-metaD-tax.biom \
--observation-metadata-fp ../data/LibCombined/full-diversity_OTU_table/taxonomy.tsv \
--sc-separated taxonomy \
--observation-header OTUID,taxonomy

In [8]:
!biom summarize-table \
-i ../data/LibCombined/full-diversity_OTU_table/feature-table-metaD-tax.biom \
-o ../data/LibCombined/full-diversity_OTU_table/feature-table-metaD-tax-summary.txt

In [9]:
!head -20 ../data/LibCombined/full-diversity_OTU_table/feature-table-metaD-tax-summary.txt

Num samples: 1,106
Num observations: 45,559
Total count: 29,008,631
Table density (fraction of non-zero values): 0.010

Counts/sample summary:
 Min: 0.000
 Max: 172,818.000
 Median: 22,873.500
 Mean: 26,228.419
 Std. dev.: 20,503.031
 Sample Metadata Categories: None provided
 Observation Metadata Categories: taxonomy

Counts/sample detail:
Lib1_Ln1_19UW-WB-08-10-O-SI: 0.000
Lib1_Ln1_19UW-WB-18-05-A-LIn-redo: 0.000
Lib1_Ln2_19UW-WB-08-10-O-SI: 0.000
Lib1_Ln2_19UW-WB-18-05-A-LIn-redo: 0.000
Lib2_Ln1_19UW-WB-06-08-O-pb-cDNA-REDO: 0.000


In [10]:
!biom convert \
-i ../data/LibCombined/full-diversity_OTU_table/feature-table-metaD-tax.biom \
-o ../data/LibCombined/full-diversity_OTU_table/feature-table-metaD-tax_json.biom \
--table-type="OTU table" \
--to-json

In [11]:
!ls ../data/LibCombined/full-diversity_OTU_table/

dna-sequences.fasta                 feature-table-metaD.biom
feature-table-metaD-tax-summary.txt feature-table.biom
feature-table-metaD-tax.biom        taxonomy.tsv
feature-table-metaD-tax_json.biom   temporary-metadata.csv


In [12]:
!qiime phylogeny align-to-tree-mafft-fasttree \
  --i-sequences ../data/LibCombined/full-diversity-rep-seqs.qza \
  --o-alignment ../data/LibCombined/aligned-rep-seqs.qza \
  --o-masked-alignment ../data/LibCombined/masked-aligned-rep-seqs.qza \
  --o-tree ../data/LibCombined/unrooted-tree.qza \
  --o-rooted-tree ../data/LibCombined/rooted-tree.qza

^C

Aborted!


In [13]:
!qiime tools export \
  --input-path ../data/LibCombined/unrooted-tree.qza \
  --output-path ../data/LibCombined/exported-tree

Exported ../data/LibCombined/unrooted-tree.qza as NewickDirectoryFormat to directory ../data/LibCombined/exported-tree
